This notebook is used to parse crsp data that only includes stocks that are ordinary/common shares and listed on the NYSE, AMEX, or NASDAQ. Also, we set negative prices equal to 'NA' since a negative price represents the midpoint of the bid-ask spread and we only want closing price values.

In [50]:
import pandas as pd

In [51]:
data = pd.read_csv("crsp_1926_2020.csv")

In [52]:
data.date = pd.to_datetime(data.date)
#data.set_index('date', inplace=True)
data = data.drop([0])

In [53]:
# clean out instrumenents that are not stock
data_to_drop = []
for index in range(1, len(data['SHRCD'])):
    curr_type = data['SHRCD'][index]
    if curr_type != 10 and curr_type != 11:
        data_to_drop.append(index)
data = data.drop(data_to_drop)

# re index the dataframe
new_index = range(len(data['SHRCD']))
data = data.reindex(new_index)

In [54]:
# clean out data to only have nyse, amex, and nasdaq 
data_to_drop = []
for index in range(0, len(data['EXCHCD'])):
    curr_type = data['EXCHCD'][index]
    if curr_type != 1 and curr_type != 2 and curr_type != 3:
        data_to_drop.append(index)
data = data.drop(data_to_drop)

# re index the dataframe
new_index = range(len(data['EXCHCD']))
data = data.reindex(new_index)
data = data.drop([0])

In [56]:
for index in range(1, len(data['PRC'])):
    if data['PRC'][index] < 0:
        data['PRC'][index] = 'NA'
    


/var/folders/db/mg0z4ws13p59206165m2snhh0000gn/T/ipykernel_42000/4101745834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PRC'][index] = 'NA'


In [58]:
data = data.dropna()
data.set_index('date', inplace=True)